## Imports

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import zipfile
import shutil
import random
import pandas as pd
import csv
import os

## Data loading

In [3]:
# Paths to training and testing data
train_data_path = '/kaggle/input/intel-image-classification/seg_train/seg_train/'
test_data_path = '/kaggle/input/intel-image-classification/seg_test/seg_test/'

# Output directories for models and reports
output_dir = '/kaggle/working'
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

## Data preprocessing

In [4]:
#Set data augmentation techniques
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.2,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1
                                                             ,channel_shift_range=50
                                                             ,brightness_range=(0,1.2)
                                                             ,preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

In [5]:
# Loading images directly from directories
batch_size = 70
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
validation_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


## All the Evaluation parameters which are used

In [6]:
import keras
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K

# Helper function to calculate F1 Score, Sensitivity, and Specificity
def calculate_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    precisions, recalls, f1_scores, specificities = [], [], [], []

    for i in range(cm.shape[0]):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - (tp + fn + fp)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1_score)
        specificities.append(specificity)

    return {
        'f1_score': np.mean(f1_scores),
        'sensitivity': np.mean(recalls),
        'specificity': np.mean(specificities)
    }

# Custom callback to save metrics and checkpoints
class SaveMetricsAndCheckpoints(keras.callbacks.Callback):
    def __init__(self, validation_data, log_file_path='/kaggle/working/metrics_log.csv', save_interval=10):
        super().__init__()
        self.validation_data = validation_data
        self.log_file_path = log_file_path
        self.save_interval = save_interval

        # Initialize the CSV file with headers
        with open(self.log_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                'epoch',
                'train_loss', 'train_accuracy', 'train_precision', 'train_recall', 
                'train_top_1_accuracy', 'train_top_5_accuracy',
                'val_loss', 'val_accuracy', 'val_precision', 'val_recall',
                'val_top_1_accuracy', 'val_top_5_accuracy',
                'f1_score', 'sensitivity', 'specificity'
            ])
        print(f"Metrics will be logged to: {self.log_file_path}")

    def on_epoch_end(self, epoch, logs=None):
        # Extract validation data
        val_images, val_labels = self.validation_data
        y_pred = self.model.predict(val_images)
        y_pred_labels = tf.argmax(y_pred, axis=1).numpy()
        y_true_labels = tf.argmax(val_labels, axis=1).numpy()

        # Calculate additional metrics
        additional_metrics = calculate_metrics(y_true_labels, y_pred_labels)

        # Save metrics to the CSV file
        with open(self.log_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                epoch + 1,
                logs.get('loss'), logs.get('accuracy'), logs.get('precision'), logs.get('recall'),
                logs.get('top_1_accuracy'), logs.get('top_5_accuracy'),
                logs.get('val_loss'), logs.get('val_accuracy'), logs.get('val_precision'), logs.get('val_recall'),
                logs.get('val_top_1_accuracy'), logs.get('val_top_5_accuracy'),
                additional_metrics['f1_score'], additional_metrics['sensitivity'], additional_metrics['specificity']
            ])
        
        print(f"Epoch {epoch + 1}: Metrics logged.")

        # Save model checkpoint every nth epoch
        if (epoch + 1) % self.save_interval == 0:
            checkpoint_filepath = (
                f"/kaggle/working/model-"
                f"{epoch + 1:02d}-"
                f"val_acc_{logs['val_accuracy']:.4f}.keras"
            )
            self.model.save(checkpoint_filepath)
            print(f"Model checkpoint saved at: {checkpoint_filepath}")

# Validation data
validation_data = next(iter(validation_generator))

# Initialize callback
metrics_and_checkpoint_callback = SaveMetricsAndCheckpoints(
    validation_data=(validation_data[0], validation_data[1]),
    log_file_path='/kaggle/working/metrics_log.csv',
    save_interval=10
)


Metrics will be logged to: /kaggle/working/metrics_log.csv


## Model design and compilation

In [7]:
keras.backend.clear_session() #clear backend
shape=(224,224,3)
input_tensor=keras.Input(shape=shape)
base_model=keras.applications.<model_name>(input_tensor=input_tensor,weights=None,include_top=False)
# Replace the model name with the models which you want to evaluate DenseNet169, ResNet50, EfficientNetB0, Xception and MobileNetV2
avg=keras.layers.AveragePooling2D(2,padding='valid')(base_model.output)
depth=keras.layers.DepthwiseConv2D(3,
                                      depthwise_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                                      bias_initializer=keras.initializers.Zeros(),depthwise_constraint=keras.constraints.NonNeg())(avg)
flat=keras.layers.Flatten()(depth)
preds=keras.layers.Dense(6,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(flat)
model=keras.Model(inputs=base_model.input, outputs=preds)  

##################################
for layer in model.layers:
  layer.trainable = True
#Determine adaptive learning rate with an initialization value of 0.045 and decay of 0.94 every two epochs.
lr_schedule =keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.045,
    decay_steps=2*int(len(train_generator.filenames)/batch_size),
    decay_rate=0.94,
    staircase=True)
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=lr_schedule)
# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
        tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')
    ]
)

## Model Running

In [8]:
hist=model.fit(
    train_generator, 
    epochs=220,
    validation_data=validation_generator,
    shuffle=True,
    callbacks=metrics_and_checkpoint_callback) #start training


Epoch 1/220


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734247552.032073     117 service.cc:145] XLA service 0x7df7e00026c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734247552.032124     117 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734247552.032128     117 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734247607.778634     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/steptep - accuracy: 0.2149 - loss: 1.7382 - precision: 0.4629 - recall: 0.0119 - top_1_accuracy: 0.2149 - top_5_accuracy: 0.8
Epoch 1: Metrics logged.
201/201 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.2152 - loss: 1.7376 - precision: 0.4645 - recall: 0.0121 - top_1_accuracy: 0.2152 - top_5_accuracy: 0.8761 - val_accuracy: 0.1843 - val_loss: 2.3925 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_top_1_accuracy: 0.1843 - val_top_5_accuracy: 0.8420
Epoch 2/220
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step/step - accuracy: 0.3847 - loss: 1.4008 - precision: 0.7642 - recall: 0.1073 - top_1_accuracy: 0.3847 - top_5_accuracy: 0.96
Epoch 2: Metrics logged.
201/201 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.3847 - loss: 1.4007 - precision: 0.7641 - recall: 0.1074 - top_1_accuracy: 0.3847 - top_5_accuracy: 0.9685 - val_accuracy: 0.3780 - val_loss: 1.4979 - val_precision: 0.5584 - val_recall: 0.0940 - val_top_1_accuracy: 0.3780 - val_top_5_accuracy: 0

KeyboardInterrupt: 

## Command to save model in your kaggle storage

In [1]:
import kagglehub

kagglehub.login()

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = 'path to the last saved checkpoint'

MODEL_SLUG = 'model_slug' # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = 'default' # Replace with variation slug.

kagglehub.model_upload(
  handle = f"update it for  your kaggle ID",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'update date ')

Uploading Model https://www.kaggle.com/models/offshimar/RES/keras/default ...


ConnectionError: HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/models/offshimar/RES/get (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7c9cd5796bf0>: Failed to resolve 'www.kaggle.com' ([Errno -3] Temporary failure in name resolution)"))

## command to restart the tarining from where you left

In [6]:
import tensorflow as tf
# Path to the checkpoint and metrics log
checkpoint_path = 'path to the last saved checkpoint '
# Load the model from the checkpoint
model = keras.models.load_model(checkpoint_path)

validation_data = next(iter(validation_generator))
# Resume training from epoch 180
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=220,  # Train for the remaining epochs
    callbacks=[metrics_and_checkpoint_callback],
    initial_epoch=#last run checkpoint value
)


Epoch 171/220


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734113732.273861     103 service.cc:145] XLA service 0x7968d40025a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734113732.273921     103 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734113732.273927     103 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734113855.182263     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 34s 8s/step ep - accuracy: 0.9462 - loss: 0.1449 - precision: 0.9536 - recall: 0.9400 - top_1_accuracy: 0.9462 - top_5_accuracy: 0.9
Epoch 171: Metrics logged.
201/201 ━━━━━━━━━━━━━━━━━━━━ 600s 2s/step - accuracy: 0.9462 - loss: 0.1449 - precision: 0.9536 - recall: 0.9400 - top_1_accuracy: 0.9462 - top_5_accuracy: 0.9998 - val_accuracy: 0.8730 - val_loss: 0.4512 - val_precision: 0.8785 - val_recall: 0.8677 - val_top_1_accuracy: 0.8730 - val_top_5_accuracy: 0.9990
Epoch 172/220
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step/step - accuracy: 0.9491 - loss: 0.1370 - precision: 0.9562 - recall: 0.9431 - top_1_accuracy: 0.9491 - top_5_accuracy: 0.9
Epoch 172: Metrics logged.
201/201 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.9491 - loss: 0.1370 - precision: 0.9562 - recall: 0.9431 - top_1_accuracy: 0.9491 - top_5_accuracy: 0.9999 - val_accuracy: 0.8753 - val_loss: 0.4523 - val_precision: 0.8807 - val_recall: 0.8710 - val_top_1_accuracy: 0.8753 - val_top_5_accuracy: 0.99